# ch04. word2vec 속도 개선
목표: word2vec 속도 개선

## 4.1 word2vec 개선 1️⃣
CBOW 모델: 거대한 말뭉치를 다루게 되면 문제 발생함
- 입력층의 원핫
    - 어휘 수가 많아지면 원핫 표현의 벡터 크기도 커짐
    - 해결) `Embedding` 계층 도입
- 은닉층 이후의 계산
    - 해결)  `Negative Sampling` 이라는 새로운 손실함수 도입
  

### 4.1.1 Embedding 계층
 `Embedding 계층`: 가중치 매개변수로부터 '단어 ID'에 해당하는 행(벡터)를 추출하는 계층


### 4.1.2 Embedding 계층 구현
행렬에서 특정 행을 추출하기는 아주 쉬움
W에서 특정행 추출하려면? W[2] or W[5]와 같이 명시하기만 하면 됨.

In [6]:
import numpy as np

W = np.arange(21).reshape(7, 3)
print(W)
print("-------------")
print("W[2]: \n", W[2])
print("W[5]: \n", W[5])

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]
-------------
W[2]: 
 [6 7 8]
W[5]: 
 [15 16 17]


In [8]:
idx = np.array([1,0, 3, 0])
print(W[idx])

[[ 3  4  5]
 [ 0  1  2]
 [ 9 10 11]
 [ 0  1  2]]


In [1]:
import numpy as np

class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0

        np.add.at(dW, self.idx, dout)
        # or
        # for i, word_id in enumerate(self.idx):
        #     dW[word_id] += dout[i]

        return None


Embedding 계층의 역전파: dh의 각 행의 값을 dW의 해당 행에 더해줘야 함.


## 4.2 word2vec 개선 2️⃣
은닉층 이후의 처리..
-> 병목 어떻게 개선? **네거티브 샘플링** 을 이용해서.
softmax 대신 네거티브 샘플링을 이용하면 어휘가 아무리 많아져도 계산량을 낮은 수준에서 일정하게 억제할 수 있음


### 4.2.1 은닉층 이후 계산의 문제점
- 은닉층의 뉴런과 가중치 행렬(W_out)의 곱
- Softmax 계층의 계산

=> 어휘 수에 비례해 계산 증가
### 4.2.2 다중 분류에서 이진 분류로
- 네거티브 샘플링 기법의 핵심
 - **이진분류** : 다중 분류를 이진 분류로 근사하는 것이 네거티브 샘플링을 이해하는데 중요함
    - 은닉층과 출력 측의 가중치 행렬의 내적은 "say"에 해당하는 열만을 추출
    - 그 추출된 벡터와 은닉층 뉴런과의 내적을 계산하면 끝


### 4.2.3 시그모이드 함수와 교차 엔트로피 오차
이진 분류 -> 신경망
- 점수에 **시그모이드 함수**를 적용해 확률로 변환
- 손실을 구할 때에는 손실 함수로 '**교차엔트로피 오차**'를 사용
![](img/4-10.png)

    - y는 신경망이 출력한 확률
    - t: 정답 레이블
    - y-t: 두 값의 차이
    - 정답 레이블이 1이라면 y가 1에 가까워질수록 오차가 줄어든다.
    - 오차가 앞 계층으로 흘러가므로, 오차가 크면 '크게' 학습하고, 작으면 '작게' 학습하게 된다.

### 4.2.4 다중 분류에서 이진 분류로 (구현)
![](img/4-11.png)
- 맥락이 "you", "goodbye"
- target: "say"
- 입력층에서 각각에 대응하는 단어 ID의 분산 표현을 추출하기 위해 Embedding 계층 사용

![](img/4-13.png)

## 4.4 word2vec 남은 주제
### 4.4.1 word2vec을 사용한 애플리케이션의 예
자연어 처리 분야에서 단어의 분산 표현이 중요한 이유: **전이 학습**
- 전이학습: 한 분야에서 배운 지식을 다른 분야에도 적용하는 기법
- 단어를 고정 길이 벡터로 변환해준다
- 문장(단어의 흐름)도 단어의 분산 표현을 사용하여 고정 길이 벡터로 변환할 수 있다.
    - 문장의 각 단어를 분산 표현으로 변환하고 그 합을 구하는 것 : `bag-of-words`. 단어의 순서를 고려하지 X

=> 단어 / 문장을 고정 길이 벡터로 변환할 수 있다는 점은 매우 중요 -> 머신러닝 기법을 적용할 수 있기 때문


### 4.4.2 단어 벡터 평가 방법
단어의 분산 표현의 우수성을 평가하는 척도
- 유사성 (ex. cat & animal // cat & car)
- 유추 문제 (ex. king:queen = man: ?)
    * 단어의 의미나 문법적인 문제를 제대로 이해하고 있는지를 (어느 정도) 측정할 수 있음.


## 4.5 정리
<word2vec 고속화>
- 개선해야 하는 이유: 말뭉치의 어휘 수 증가에 비례해 계산량 증가
- `Embedding 계층`
    - 단어의 분산 표현을 담고 있으며, 순전파 시 지정한 단어 ID의 행(벡터)를 추출
- `Negative Sampling`
    - 부정적인 예를 몇 개 샘플링하는 기법, 이를 사용하면 다중 분류를 이진 분류처럼 취급 가능
- => 핵심) '모두' 대신 **일부** 만을 처리.
    - 계산을 효율적으로

- word2vec은 전이 학습 측면에서 특히 중요하며, 그 단어의 분산 표현은 다양한 자연어 처리 작업에 이용될 수 있다. 